<font size=7 face="Courier">Uploader App - Source Code

<font color="red">

**To Do**
* Check if selected UUID is already up on PRP

<font color="orange">Folder used for testing: `/home/mxwbio/Desktop/MxW/SampleDataSet/Organoids/5167/Activity_Scan/000003/`

# <font color="gray"> Set Up Notebook

import packages

In [ ]:
import ipywidgets as ipw

In [ ]:
from IPython.core.display import HTML, display, Javascript, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
import json
import h5py
import os
import glob
import copy
import sys
import subprocess

import braingeneers.utils.s3wrangler as wr
from braingeneers.utils import smart_open
#from PyQt5.QtWidgets import QApplication, QWidget, QMessageBox, QLineEdit, QFileDialog, QPushButton, QLabel, QTextEdit
#from PyQt5.QtGui import QFont
#from PyQt5.QtGui import QIcon

Set up variables for S3

In [ ]:
s3 = 'aws --endpoint https://s3.nautilus.optiputer.net s3'
s3_path = 'braingeneers/ephys/'

Set empty object to hold everything

In [ ]:
Uploader = type('uploader', (object,), {})()

# <font color="blue">Functions

## <font color="blue">Get Uploadable files in selected folder

In [ ]:
def getExperiments(exp_path=''):
    print(exp_path)
    exp_list = glob.glob(exp_path + '/' + '*.h5')
    for i, exp in enumerate(exp_list):

        try:
            with h5py.File(exp, "r") as f:
                pass

        except OSError:
            #msg = QMessageBox()
            #msg.setIcon(QMessageBox.Warning)

            print(f'Error: The file {exp} could not be read!')
            #msg.setWindowTitle("Warning")
            #msg.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
            #retval = msg.exec_()
            #exp_list.remove(exp)

    return exp_list

## <font color="blue">Generate UUID for experiment

In [ ]:
def get_uuid_prefix(dir_name, exp_list):
    f = h5py.File(exp_list[0], "r")
    keys = list(f.keys())
    time_stamp = str(f.get('time')[()])

    # Correct format
    meta_time_stamp = time_stamp[10:20] + 'T' + time_stamp[21:29]
    f.close()

    uuid = meta_time_stamp[:10] + '-e-'
    return uuid


## <font color="blue"> Create Template for metadata file

In [ ]:
def get_metadata_exp_template():
    exp_metadata = {
        "blocks": [ {"num_frames": 0, "path": "", "source": "", "timestamp": "" } ],
        "channels": [],
        "hardware": "Maxwell",
        "name": "1well-maxwell",
        "notes": "",
        "num_channels": 0,
        "num_current_input_channels": 0,
        "num_voltage_channels": 0,
        "offset": 0,
        "sample_rate": 20000,
        "scaler": 1,
        "timestamp": "",
        "units": "\u00b5V",
        "version": "0.0.1"
    }
    return exp_metadata

## <font color="blue">Generate Metadata from h5 files

In [ ]:

def gen_metadata(dir_name, exp_list, uuid='', notes=''):
    # Open file
    with h5py.File(exp_list[0], "r") as f:
        keys = list(f.keys())
        time_stamp = str(f.get('time')[()])

        # Correct format
        meta_time_stamp = time_stamp[10:20] + 'T' + time_stamp[21:29]

    metadata = {
        "experiments": [
        ],
        "notes": " ",
        "timestamp": "",
        "uuid": "YYYY-MM-DD-[e]-[descriptor]"
    }

    metadata['timestamp'] = meta_time_stamp
    metadata['uuid'] = uuid
    metadata['notes'] = notes
    metadata['experiments'] = [f"experiment{i + 1}.json" for i in range(len(exp_list))]

    with open(dir_name + '/' + "metadata.json", "w") as outfile:
        json.dump(metadata, outfile, indent=2)

## <font color="blue">Generate metadata for each experiment in selected folder

In [ ]:

def gen_metadata_exp(dir_name, exp_list):
    '''
    Loop through each experiment file and save corresponding metadata file
    '''
    exp_metadata = get_metadata_exp_template()

    for i, exp in enumerate(exp_list):

        print(i, exp)

        try:
            with h5py.File(exp, "r") as f:
                cur_metadata = copy.copy(exp_metadata)

                # Get keys
                keys = list(f.keys())
                time_stamp = str(f.get('time')[()])
                # Correct time format
                meta_time_stamp = time_stamp[10:20] + 'T' + time_stamp[21:29]

                # Get blocks
                shape = f.get('sig').shape
                num_frames = shape[0] * shape[1]
                print('Total frames * channels', num_frames)

                # Get path
                data_path = exp

                # Set values
                cur_metadata['blocks'][0]['num_frames'] = num_frames
                cur_metadata['blocks'][0]['path'] = exp
                cur_metadata['blocks'][0]['timestamp'] = meta_time_stamp
                cur_metadata['num_channels'] = shape[0]
                cur_metadata['num_voltage_channels'] = shape[0]
                cur_metadata['timestamp'] = meta_time_stamp

                # Write file
                with open(dir_name + '/' + "experiment{}.json".format(i + 1), "w") as outfile:
                    json.dump(cur_metadata, outfile, indent=2)

        except OSError:
            msg = QMessageBox()
            msg.setIcon(QMessageBox.Warning)

            msg.setText(f'The file {exp} could not be read!')
            msg.setWindowTitle("Warning")
            msg.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
            retval = msg.exec_()

## <font color="blue">Upload Data to S3

In [ ]:
def uploadToS3(uuid, dir_name, exp_list):
    # cmd = f"{s3} sync {dir_name} s3://{s3_path + uuid} "
    # process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # output, error = process.communicate()

    # Upload metadata
    wr.upload(local_file=f'{dir_name}/metadata.json', path='s3://braingeneers/ephys/' + uuid + '/metadata.json')

    for i in range(len(exp_list)):
        wr.upload(local_file=dir_name + f'/experiment{i + 1}.json',
                  path=f's3://braingeneers/ephys/{uuid}/original/experiment{i + 1}.json')

    # Upload data files
    for i,exp in enumerate(exp_list):
        print(f'Uploading file {i + 1}/{len(exp_list)}...')
        print(f'Uploading {exp}')
        filename = exp.split('/')[-1]
        print(f'To: s3://braingeneers/ephys/{uuid}/original/data/{filename}')

        b = None
        file_size = os.path.getsize(exp)/1000000
        megabyte_count = 0
        with open(exp, 'rb') as local_file, smart_open.open( f's3://braingeneers/ephys/'\
                                                            f'{uuid}/original/data/{filename}', 'wb') as s3file:
            while b != b'':
                b = local_file.read(1000000)
                s3file.write(b)
                megabyte_count += 1
                print(f'{megabyte_count}\t of \t {file_size:3.1f} MB')
                #app.main_app.processEvents()

        # wr.upload(local_file=exp, path=f's3://braingeneers/ephys/{uuid}/original/data/{filename}')

    print(" ")
    print('Upload SuccessfullyComplete!')

# <font color="green">Widgets

## <font color="green">Start Button

In [ ]:
wStart = ipw.Button(description="Start", button_style="success", layout=ipw.Layout(width='auto'))
wStart

## <font color="green">Restart Button

In [ ]:
wRestart = ipw.Button(icon="refresh", button_style="danger", layout=ipw.Layout(width='auto'))
wRestart

## <font color="green">Continue to Check Data

Button that allows uster to Checks data after the have selected a folder with data

In [ ]:
wContinueCheckData = ipw.Button(description="Continue", button_style="info", layout=ipw.Layout(width='auto'))
wContinueCheckData 

## <font color="green"> Check Data Error Message

The html text is used to display message to the user if there is an issue with the data or folder they selected

In [ ]:
wFolderCheckMsg = ipw.HTML("<h4 class='text-danger'>Select a Folder to Continue</h4>",  layout=ipw.Layout(visibility = "hidden") )
wFolderCheckMsg

## <font color="green"> Continue to Metadata Setup

After users data is confirmed to be uploadable continue to metadata setup

In [ ]:
wContinueMetadata = ipw.Button(description="Continue", button_style="info", layout=ipw.Layout(width='auto')) # disabled=True,
wContinueMetadata

## <font color="green">Select Directory to Upload

<font color="green">wSelectFolder</font> is used to select the folder on the maxwell computer with data that we want to uploade. 

This widget makes use of [ipyfilechooser](https://pypi.org/project/ipyfilechooser/)

In [ ]:
from ipyfilechooser import FileChooser
wFileChooser = FileChooser('/home/mxwbio/wetai/homepage')
wFileChooser.show_only_dirs = True
wFileChooser

## <font color="green">UUID Text- Dataset Name

This textbox is where the user inputs their unique id for the experiment, the `UUID`

In [ ]:
wUUID = ipw.Text(description="UUID")
wUUID

## <font color="green">Notes Textbox

User can type additional notes here that wil be added to the metadata

In [ ]:
wNotes=ipw.Textarea()
wNotes

## <font color="green">Upload Dataset Button

This button will upload the dataset to the braingeneers PRP

In [ ]:
wUpload = ipw.Button(description="Upload", button_style="success", layout=ipw.Layout(width='auto'))
wUpload

# <font color="orchid">Final App

We combine the widgets and functions to build an interactive app.

## <font color="orchid"> Start App

In [ ]:
def showFolderSelector(b):
    clear_output()
    print("Select Folder to Upload Data From")
    print(" ")
    display(wFileChooser)
    display(wFolderCheckMsg)
    display(wContinueCheckData)
    print(" ")
    display(wRestart)
    
wStart.on_click( showFolderSelector )

## <font color="orchid"> Restart App

In [ ]:
def restartApp(b):
    clear_output()
    wFileChooser.reset()                          # Resets the file choos widget
    wFolderCheckMsg.layout.visibility = "hidden"   # Reset the warning message
    wNotes.value = ""
    display(wStart)                               # Go back to start button
    
wRestart.on_click( restartApp )

## <font color="orchid">Check data in selected folder

In [ ]:
def checkData(b):
    if wFileChooser.value == None:     # Check that user selected a folder before continuing
        wFolderCheckMsg.layout.visibility = "visible"
    else:
        clear_output()
        print('Checking Data in:')
        Uploader.exp_list = getExperiments(wFileChooser.value[:-1])
        print(" ")
        print("The following "+str(len(Uploader.exp_list))+" files will be uploaded")
        for i in Uploader.exp_list:
            print(i)
        print(" ")
        #wContinueMetadata.disabled=False
        display(wContinueMetadata)
        print(" ")
        display(wRestart)
        
wContinueCheckData.on_click(checkData)

## <font color="orchid">Name Dataset

In [ ]:
def nameData(b):
    wUUID.value = get_uuid_prefix(wFileChooser.value,Uploader.exp_list)+"NAME_HERE" 
    
    clear_output()
    print("Name your Dataset")
    display(wUUID)
    print("Additional Notes")
    display(wNotes)
    display(wUpload)
    print(" ")
    display(wRestart)
    
wContinueMetadata.on_click(nameData)

## <font color="orchid">Upload Dataset

In [ ]:
def upload(b):
    clear_output()
    print("Generating Metadata...")
    print(" ")
    gen_metadata(wFileChooser.value[:-1], Uploader.exp_list, uuid=wUUID.value, notes=wNotes.value)
    gen_metadata_exp(wFileChooser.value[:-1], Uploader.exp_list)
    print(" ")
    print("Uploading Data...")
    print(" ")
    
    uploadToS3( wUUID.value, wFileChooser.value[:-1], Uploader.exp_list)
    print(" ")
    display(wRestart)
    
wUpload.on_click(upload)

## <font color="orchid">Test App

Uncomment the line below to run the app and test that it works. When test I recommend you `Cell -> Run All` code above. This is useful for debugging.

In [ ]:
wStart

# <font color="brown">Scratch Paper

## <font color="brown"> Old Code

### Initialization code

In [ ]:
# class App(QWidget):

#     def __init__(self,main_app = None):
#         super().__init__()
#         self.main_app = main_app
#         self.title = 'MaxWell S3 Experiment Upload'
#         self.left = 10
#         self.top = 10
#         self.width = 800
#         self.height = 480

#         self.button1 = None
#         self.button2 = None
#         self.button3 = None

#         self.label1 = None
#         self.label2 = None
#         self.label3 = None
#         self.label4 = None
#         self.label5 = None

#         self.textbox1 = None
#         self.textboxlabel1 = None
#         self.textbox2 = None
#         self.textboxlabel2 = None

#         self.infobutton = None

#         self.dir_name = 'No Directory Selected'
#         self.exp_list = ['No experiments chosen yet']
#         self.uuid_prefix = ''

#         self.init_ui()

### Creating widgets

In [ ]:
#     def init_ui(self):
#         self.setWindowTitle(self.title)
#         self.setGeometry(self.left, self.top, self.width, self.height)

#         self.button1 = QPushButton(self)
#         self.button1.move(0, 5)
#         self.button1.setText("Open Directory")
#         self.button1.clicked.connect(self.button1_clicked)

#         self.button2 = QPushButton(self)
#         self.button2.move(0, 60)
#         self.button2.setText("Generate Metadata")
#         self.button2.clicked.connect(self.button2_clicked)
#         self.button2.setEnabled(False)

#         self.button3 = QPushButton(self)
#         self.button3.move(0, 360)
#         self.button3.setText("Upload")
#         self.button3.clicked.connect(self.button3_clicked)
#         self.button3.setEnabled(False)

#         self.infobutton = QPushButton(self)
#         self.infobutton.move(140, 360)
#         self.infobutton.setText("Help")
#         self.infobutton.clicked.connect(self.infobutton_clicked)

#         self.label1 = QLabel(self)
#         self.label1.setFixedWidth(640)
#         self.label1.move(20, 40)
#         self.label1.setText(self.dir_name)

#         self.label2 = QLabel(self)
#         # self.label2.setFixedWidth(200)
#         # self.label2.setFixedHeight(20)

#         self.label2.move(220, 20)
#         self.label2.resize(600, 400)
#         self.label2.setText('\n'.join(self.exp_list))

#         self.label3 = QLabel(self)
#         self.label3.setFixedWidth(400)
#         self.label3.move(220, 0)
#         font = self.label3.font()
#         font.setBold(True)
#         self.label3.setFont(font)
#         self.label3.setText('Experiments:')

#         self.label4 = QLabel(self)
#         self.label4.setFixedWidth(400)
#         self.label4.move(20, 400)
#         self.label4.setText('')

#         self.label5 = QLabel(self)
#         self.label5.setFixedWidth(400)
#         self.label5.move(20, 420)
#         self.label5.setText('')

#         self.textboxlabel1 = QLabel(self)
#         self.textboxlabel1.setFixedWidth(400)
#         self.textboxlabel1.move(20, 90)
#         self.textboxlabel1.setText('UUID:')

#         self.textbox1 = QLineEdit(self)
#         self.textbox1.move(20, 110)
#         self.textbox1.resize(180, 20)
#         self.textbox1.setEnabled(False)
#         self.textbox1.setText("None")

#         self.textboxlabel2 = QLabel(self)
#         self.textboxlabel2.setFixedWidth(400)
#         self.textboxlabel2.move(20, 130)
#         self.textboxlabel2.setText('Notes:')

#         self.textbox2 = QTextEdit(self)
#         self.textbox2.move(20, 150)
#         self.textbox2.resize(180, 200)
#         self.textbox2.setEnabled(False)
#         self.textbox2.setText("")



#         self.show()

### <font color="orange">Adding function to widgets

In [ ]:
#     def button1_clicked(self):
#         self.open_directory_dialog()
#         self.button2.setEnabled(True)
#         self.uuid_prefix = get_uuid_prefix(self.dir_name, self.exp_list)
#         self.textbox1.setText(self.uuid_prefix)
#         self.textbox1.setEnabled(True)
#         self.textbox2.setEnabled(True)

#     def button2_clicked(self):
#         print('Generating metadata...')

#         self.uuid = self.textbox1.text()
#         self.notes = self.textbox2.toPlainText()

#         gen_metadata(self.dir_name, self.exp_list, uuid=self.uuid, notes=self.notes)
#         gen_metadata_exp(self.dir_name, self.exp_list)

#         self.button3.setEnabled(True)

#     def button3_clicked(self):
#         print('Uploading to S3...')
#         self.button3.setText("Uploading...")
#         upload_dir_s3(self.uuid, self.dir_name, self.exp_list,app=self)

#         # self.label4 = QLabel(self)
#         # self.label4.setFixedWidth(400)
#         # self.label4.move(0, 380)
#         # self.label4.setText('COMPLETE')




#     def open_directory_dialog(self):
#         options = QFileDialog.Options()
#         options |= QFileDialog.DontUseNativeDialog
#         self.dir_name = str(
#             QFileDialog.getExistingDirectory(self, "Experiment Directory for S3 upload", options=options))

#         self.label1.setText(self.dir_name)

#         self.exp_list = get_experiments(self.dir_name)
#         self.label2.setText('\n'.join(self.exp_list))

### Functions

In [ ]:
# Methods for creating metadata
# def get_experiments(exp_path=''):
#     print(exp_path)
#     exp_list = glob.glob(exp_path + '/' + '*.h5')
#     for i, exp in enumerate(exp_list):

#         try:
#             with h5py.File(exp, "r") as f:
#                 pass

#         except OSError:
#             msg = QMessageBox()
#             msg.setIcon(QMessageBox.Warning)

#             msg.setText(f'The file {exp} could not be read!')
#             msg.setWindowTitle("Warning")
#             msg.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
#             retval = msg.exec_()
#             exp_list.remove(exp)

#     return exp_list


# def get_uuid_prefix(dir_name, exp_list):
#     f = h5py.File(exp_list[0], "r")
#     keys = list(f.keys())
#     time_stamp = str(f.get('time')[()])

#     # Correct format
#     meta_time_stamp = time_stamp[10:20] + 'T' + time_stamp[21:29]
#     f.close()

#     uuid = meta_time_stamp[:10] + '-e-'
#     return uuid


# def gen_metadata(dir_name, exp_list, uuid='', notes=''):
#     # Open file
#     with h5py.File(exp_list[0], "r") as f:
#         keys = list(f.keys())
#         time_stamp = str(f.get('time')[()])

#         # Correct format
#         meta_time_stamp = time_stamp[10:20] + 'T' + time_stamp[21:29]

#     metadata = {
#         "experiments": [
#         ],
#         "notes": " ",
#         "timestamp": "",
#         "uuid": "YYYY-MM-DD-[e]-[descriptor]"
#     }

#     metadata['timestamp'] = meta_time_stamp
#     metadata['uuid'] = uuid
#     metadata['notes'] = notes
#     metadata['experiments'] = [f"experiment{i + 1}.json" for i in range(len(exp_list))]

#     with open(dir_name + '/' + "metadata.json", "w") as outfile:
#         json.dump(metadata, outfile, indent=2)


# def gen_metadata_exp(dir_name, exp_list):
#     '''
#     Loop through each experiment file and save corresponding metadata file
#     '''
#     exp_metadata = get_metadata_exp_template()

#     for i, exp in enumerate(exp_list):

#         print(i, exp)

#         try:
#             with h5py.File(exp, "r") as f:
#                 cur_metadata = copy.copy(exp_metadata)

#                 # Get keys
#                 keys = list(f.keys())
#                 time_stamp = str(f.get('time')[()])
#                 # Correct time format
#                 meta_time_stamp = time_stamp[10:20] + 'T' + time_stamp[21:29]

#                 # Get blocks
#                 shape = f.get('sig').shape
#                 num_frames = shape[0] * shape[1]
#                 print('Total frames * channels', num_frames)

#                 # Get path
#                 data_path = exp

#                 # Set values
#                 cur_metadata['blocks'][0]['num_frames'] = num_frames
#                 cur_metadata['blocks'][0]['path'] = exp
#                 cur_metadata['blocks'][0]['timestamp'] = meta_time_stamp
#                 cur_metadata['num_channels'] = shape[0]
#                 cur_metadata['num_voltage_channels'] = shape[0]
#                 cur_metadata['timestamp'] = meta_time_stamp

#                 # Write file
#                 with open(dir_name + '/' + "experiment{}.json".format(i + 1), "w") as outfile:
#                     json.dump(cur_metadata, outfile, indent=2)

#         except OSError:
#             msg = QMessageBox()
#             msg.setIcon(QMessageBox.Warning)

#             msg.setText(f'The file {exp} could not be read!')
#             msg.setWindowTitle("Warning")
#             msg.setStandardButtons(QMessageBox.Ok | QMessageBox.Cancel)
#             retval = msg.exec_()

### Calling main app

In [ ]:
# if __name__ == '__main__':
#     main_app = QApplication(sys.argv)
#     ex = App(main_app)
#     sys.exit(main_app.exec_())

### commented out code

In [ ]:
# filename = '210626_Stim2.raw.h5'
# descriptor = 'Test'
# notes = 'Test_notes'

# with h5py.File(filename, "r") as f:
#     # List all groups
#     print("Keys: %s" % f.keys())
#     keys = list(f.keys())
#     a_group_key = list(f.keys())[0]
#     # Get the data
#     data = list(f[a_group_key])
#     x = f.get('sig')[()]
#     settings = f.get('time')[()]
#     mapping = f.get('mapping')[()]